# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'data/emotion_Summaries.pkl.bz2'
Abstracts_file = 'data/emotion_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(list)

for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

/bin/bash: pip: command not found


In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

INPUT TEXT:
  Good muffins cost $3.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/michaelmoor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(stemmer.stem("processes"))

process


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for doc_id in Summaries.keys():
    tokens = preprocess(tokenize(Summaries[doc_id].title))
    if (doc_id in Abstracts):
        tokens.extend(preprocess(tokenize(Abstracts[doc_id])))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

Let's test these functions with some examples:

In [9]:
print(tf('amsterdam', 33741990))
print(df('amsterdam'))
print(num_documents)

2.0
8.0
46483.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** Michael

### Task 1

Implement in the code block below the `smarter_tokenize_and_preprocess` function using NLTK's functions for tokenization and stemming. 

In [10]:
# Smarter linguistic processing

# Your code here:

def smarter_tokenize_and_preprocess(text):
    text = word_tokenize(text)
    stemmed_list = []
    for word in text:
        stemmed_list.append(stemmer.stem(word))
    return stemmed_list

# To test it:
print(smarter_tokenize_and_preprocess("Books about Information Retrieval (IR) etc. cost at least $15.00!"))

['book', 'about', 'inform', 'retriev', '(', 'ir', ')', 'etc', '.', 'cost', 'at', 'least', '$', '15.00', '!']


Now we can make a smarter index based on this function. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [11]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(list)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = list(key for key in Summaries.keys() if 33000000 <= key < 34000000)

# Building our smarter index:
for id in sorted(subset_of_ids):
    term_set = set(smarter_tokenize_and_preprocess(Summaries[id].title))
    if id in Abstracts:
        term_set.update(smarter_tokenize_and_preprocess(Abstracts[id]))
    for term in term_set:
        smarter_index[term].append(id)

Now implement the `smarter_and_query` function, based on the function `smarter_tokenize_and_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [12]:
def and_merge(sorted_list1, sorted_list2):
    # Initialise merged list
    merged_list = []
    # first we make copies of the lists, so we don't modify the existing lists in the index:
    list1 = list(sorted_list1)
    list2 = list(sorted_list2)
    # ---
    # write your code here...
    # (use list1 and list2 here, not sorted_list1 and sorted_list2)
    # ---
    """
    Condition that both lists must contain at least one value.
    While loop that checks if the value in list 1 is also in list 2, if this condition is met, the values are added to the merged      list and removed from their respective lists.
    If this condition is not met, check which value is the smallest and remove it from it's respective list.
    """
    while len(list1) > 0 and len(list2) > 0:
        if list1[0] == list2[0]:
            merged_list.append(list1[0])
            list1.remove(list1[0])
            list2.remove(list2[0])
        elif list1[0] < list2[0]:
            list1.remove(list1[0])
        elif list1[0] > list2[0]:
            list2.remove(list2[0])

    return merged_list

Now implement the `smarter_and_query` function, based on the function `smarter_tokenize_and_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [13]:
# Smarter and_query based on the smarter tokenize and preprocess functions

# Your code here:

def smarter_and_query(query_string):
    # Prepare terms to be used in query search.
    query_terms = smarter_tokenize_and_preprocess(query_string)
    documents = []
    # Recover all results from querying each word and add them to the documents list.
    for term in query_terms:
        query_results = []
        for i in smarter_index[term]:
            query_results.append(i)
        documents.append(query_results)

    # To prevent checking against itself, initialize merged_list as the first list in documents.
    merged_list = documents.pop(0)
    for doc_list in documents:
        # and_merge merged_list with new list in documents, update merged list with result from merge.
        merged_list = and_merge(merged_list, doc_list)
    
    return merged_list

print(smarter_and_query("automated quality controls"))
print(len(smarter_and_query("automated quality controls"))) 

[33621483, 33741990]
2


### Task 2

Run the query "automated quality controls" with the new `smarter_and_query` function from task 1. Does it return paper *33741990*? Explain what our new smarter function specifically contributes to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [14]:
# Add your code here
print(smarter_and_query("automated quality controls"))

display_summary(33741990, show_abstract=True)

[33621483, 33741990]


**Answer:** Through the use of the new smarter functions, the document 33741990, is now also selected even though it does not contain the word "controls" but "control". This is would not have been selected with the non-smart functions as it would look directly for controls and not stem like control.

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [15]:
# Your code here:

def idf(t):
   return log10((len(Summaries) + 1)/(df(t) + 1))

def tfidf(t,d):
   return (tf(t,d) * idf(t))

print(tfidf('utrecht', 31854264))
print(tfidf('rehabilitation', 31854264))
print(tfidf('with', 31854264))

3.8891522421307827
2.2094215957804337
0.5813683068568097


### Task 4

Create a function `query_ntn_nnn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn.nnn`, as above (see the formula for the `ntn.nnn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn_nnn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your functions `or_merge` and `or_query` from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "happiness and sadness of young adults".

In [26]:
# Code source: Example solutions 02

def or_merge(sorted_list1, sorted_list2):  
    merged_list = []  
    list1 = list(sorted_list1)  
    list2 = list(sorted_list2)  
    while (True):  
        if (not list1):  
            merged_list.extend(list2)  
            break  
        if (not list2):  
            merged_list.extend(list1)  
            break  
        if (list1[0] < list2[0]):  
            merged_list.append(list1[0])  
            list1.pop(0)  
        elif (list1[0] > list2[0]):  
            merged_list.append(list2[0])  
            list2.pop(0)  
        else:  
            merged_list.append(list1[0])  
            list1.pop(0)  
            list2.pop(0)  
    return merged_list

def or_query(query_string):  
    query_words = preprocess(tokenize(query_string))  
    first_word = query_words[0]  
    remaining_words = query_words[1:]  
    or_list = inverted_index[first_word]  
    for t in remaining_words:  
        or_list = or_merge(or_list, inverted_index[t])  
    return or_list  

In [30]:
# Your code here:

def score_ntn_nnn(query_words, doc_id):
   final_score = []
   for term in query_words:
       final_score.append(tfidf(term, doc_id))
   return (doc_id, sum(final_score))

def query_ntn_nnn(query_string):
   query_words = preprocess(tokenize(query_string))
   doc_ids = or_query(query_string)
   score = []
   for doc_id in doc_ids:
       score.append(score_ntn_nnn(query_words, doc_id))
   result = sorted(score, key = lambda i:i[1], reverse = True)

   return result[:10]

#Example query:
query_ntn_nnn("happiness and sadness of young adults")

[(27911349, 26.993018636884692),
 (33296388, 24.082798183614212),
 (30018584, 20.979207154432146),
 (24046755, 19.32478209989241),
 (30538661, 18.274330589392466),
 (29429569, 18.19023743345421),
 (28750544, 17.79820983199086),
 (26097459, 16.61795011524937),
 (33988083, 16.580764554299265),
 (21517168, 16.556211119394842)]

### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc_ntc`. This second version should use, as its name suggests, variant `ntc.ntc` instead of `ntn.nnn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc.nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [18]:
tfidf_length_values = defaultdict(int)

for doc_id in Summaries.keys():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc_ntc` function with the same example query as above: "evolution of resistance in the context of rice"

In [19]:
# Your code here:

#def score_ntc_ntc(query_words, doc_id):
#    ...

#def query_ntc_ntc(query_string):
#    ...

# Example query:
#query_ntc_ntc("happiness and sadness of young adults")

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.